# C.C.A.S.A.: Car Concentration And Security Assistant
Bracco Filippo & Di Vece Chiara 


## Introduzione

Stanchezza, nervosismo, distrazione e sonnolenza sono le cause di oltre il 66% degli incidenti. Questi dati allarmanti ci hanno indotto a pensare un dispositivo che possa evitare incidenti di questa natura, verificando oggettivamente che il conducente sia sempre concentrato sulla strada. Lo scopo del programma è quello di rilevare particolari condizioni  non compatibili con una guida sicura attraverso il rilevamento dei tratti del viso e, in particolar modo, degli occhi, accertandosi che lo sguardo sia rivolto verso la strada mentre il veicolo è in movimento.
In seguito risponderà adeguatamente per richiamare all’attenzione il guidatore con avvisi acustici e visivi. 
Lo scopo principale è quello di evitare molti incidenti e poter salvare molte vite. 

## Codice

Si importano le librerie:
* **CV2**: libreria open-source per la gestione di immagini e video;
* **NUMPY**: libreria per calcolo scientifico;
* **PYGLET**: libreria per la gestione di immagini, video e suoni (utilizzata in questo codice solo per questi ultimi).

In [ ]:
import cv2
import numpy as np
import pyglet

## Definizione costanti
**MISURAZIONI** e **SOGLIA** sono utilizzate per discriminare lo stato di *occhi aperti* oppure *occhi chiusi*.


In [ ]:
MISURAZIONI = 10 
SOGLIA = 10  

## Variabili globali

**stato_occhi**: è un dizionario formato da:
* la lista *'rilevamenti'*, contenente il numero di occhi rilevati nelle ultime misurazioni (la cardinalità della lista è definita dalla costante *MISURAZIONI*), iniazialmente la lista è riempita di 2 (condizione normale);
* la variabile booleana *'occhi_chiusi'* che descrive lo stato corrente degli occhi. 

In [ ]:
stato_occhi = {'rilevamenti': [2]*misurazioni, 'occhi_chiusi': False} 

## Definizione funzioni

1) La funzione **check** ha come parametro in ingresso un intero **occhi_trovati** che rappresenta il numero di occhi rilevati in un singolo frame. Lo scopo è quello di aggiornare la variabile globale *stato_occhi*: 
* viene aggiunto il valore acquisito in coda alla lista *'rilevamenti'* del dizionario;
* viene rimosso il primo elemento della suddetta lista (ovvero la rilevazione più vecchia tra quelle salvate);
* viene calcolata la somma dei rilevamenti nella lista;
* se il numero è superiore alla soglia definita, gli occhi sono aperti, quindi si assegna alla variabile *'occhi_chiusi'* del dizionario il valore *False*, *True* altrimenti.

In [ ]:
def check(occhi_trovati):
    
    stato_occhi['rilevamenti'].append(occhi_trovati)
    stato_occhi['rilevamenti'].pop(0)
    numero_occhi = sum(stato_occhi['rilevamenti'])
    
    if numero_occhi > SOGLIA:
        stato_occhi['occhi_chiusi'] = False
    else:
        stato_occhi['occhi_chiusi'] = True

2) La funzione **cambia_colore** serve a cambiare il colore di pixel specifici di un'immagine. I parametri di ingresso sono l'immagine **immagine**, una lista, **pixels**, contenente le coppie di coordinate relative ai pixel il cui valore deve essere cambiato, e tre interi, **B**, **G** e **R**, che rappresentano il valore da assegnare ai pixel dell'immagine contenuti nella lista.

In [ ]:
def cambia_colore(immagine, pixels, B, G, R): 
    for (w,k) in pixels:
        immagine[w][k] = [B, G, R]

3) La funzione **draw** serve a sovrascrivere un'immagine su un'altra più grande a partire da una precisa posizione. Essa riceve in ingresso l'immagine base **img1**, la seconda immagine **img2** e due interi, **i** e **j**, che rappresentano la coordinata dell'immagine base dalla quale iniziare a copiare la seconda.

In [ ]:
def draw(img1, img2, i, j):          #i = coordinata verticale, j = orizzontale
    h,w, _ = img2.shape
    img1[i:i+h, j:j+w] = img2[:h, :w] 

4) La funzione **main_face** ha come parametro in ingresso la lista **facce**. Essa contiene uno o più elementi, ognuno dei quali rappresenta una faccia rilevata nell'immagine in ingresso dalla camera; ogni elemento è una lista di quattro valori che caratterizzano la regione dell'immagine in cui è presente il volto, ovvero posizione orizzontale, posizione verticale, larghezza e altezza. La funzione restituisce i quattro parametri (**fx**, **fy**, **fw** e **fh**) della faccia con le dimensioni maggiori tra quelle contenute nella nella lista. 

In [ ]:
def main_face(facce):                # restituisce i parametri della faccia più estesa rilevata nell'immagine
    fx = facce[0][0]
    fy = facce[0][1]
    fw = facce[0][2]
    fh = facce[0][3]
    
    for [x, y, w, h] in facce:
        if w*h > fw*fh:
            fx = x
            fy = y
            fw = w
            fh = h
    return fx, fy, fw, fh

## Caricamento dati

Si caricano i file relativi alle funzioni di classificazione per faccia e occhi (materiale OpenCV).

In [ ]:
face_cascade = cv2.CascadeClassifier('../Assets/lib/faccia.xml')
eye_cascade = cv2.CascadeClassifier('../Assets/lib/occhiali.xml')

Si utilizza la funzione di **cv2** per consentire l'accesso alla web cam e si setta le dimensioni dell'immagine in ingresso a 320x240 pixels.

In [ ]:
cv2.VideoCapture(0)
cap = cv2.VideoCapture(0) 
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 320) 
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 240)

Si importano le immagini e i suoni utilizzati nel programma. 

In [ ]:
quadro = cv2.imread('../Assets/Img/quadro.png')
start = cv2.imread('../Assets/Img/start.png')
guida_sicura = cv2.imread('../Assets/Img/guida_sicura.png')
guida_distratta = cv2.imread('../Assets/Img/distrazione.png')
pericolo = cv2.imread('../Assets/Img/Pericolo.png')
triangolo = cv2.imread('../Assets/Img/Allarme_sonno.png')
triangolo_giallo = cv2.imread('../Assets/Img/attenzione_giallo.png')
occhi_aperti = cv2.imread('../Assets/Img/occhi_aperti.png')
sound = pyglet.media.load('../Assets/audio/avviso.mp3')
sound2 = pyglet.media.load('../Assets/audio/avviso2.mp3')

L'immagine di base sulla quale vengono sovrascritte le altre secondarie è quella del quadro:

![Quadro](../Assets/Img/quadro.png)

Si carica un file di testo contenente coppie di numeri interi che rappresentano le posizioni dei pixel del quadro da cambiare all'accensione.

In [ ]:
pixels = np.loadtxt('../Assets/files/pixels_quadro.txt', dtype = 'int')

Per creare il file si sono salvate le posizioni dei pixel del quadro con valore (70, 60, 60), che rappresenta il colore del quadro spento, in una lista, la quale è stata poi salvata su file.

![File_pixels](../Assets/Img/file_img.png)

## Inizializzazione

Viene stampata sul quadro l'immagine *start* in posizione (100, 370) dell'immagine *quadro* tramite la funzione **draw**:

In [ ]:
draw(quadro, start, 100, 370)

![Start](../Assets/Img/start.png)

Si utilizzano le funzioni di **Pyglet** per la gestione degli audio: vengono creati due oggetti audio, **avviso** e  **avviso2**, che possono essere avviati e messi in pausa nel corpo del programma.

In [ ]:
looper = pyglet.media.SourceGroup(sound.audio_format, None)          #crea looper audio (occhi chiusi)
looper.loop = True
looper.queue(sound)
avviso = pyglet.media.Player()
avviso.queue(looper)

looper2 = pyglet.media.SourceGroup(sound2.audio_format, None)          #crea looper audio 2 (distrazione)
looper2.loop = True
looper2.queue(sound2)
avviso2 = pyglet.media.Player()
avviso2.queue(looper2)

Viene definita la variabile booleana **distrazione** e assegnato il valore *False* all'inizio.

In [ ]:
distrazione = False

## Corpo codice

Inizio di un ciclo **while** che stampa a schermo (tramite apposita funzione dell'ambiente) l'immagine **quadro** (*spento*) finchè su tastiera non viene premuto il tasto '**s**'. Premuto il tasto viene inserita nel quadro l'immagine **guida sicura** e viene cambiato il colore dei pixel del quadro (accensione). ![Guida sicura](../Assets/Img/guida_sicura.png)

In [ ]:
while cv2.waitKey(30) != ord('s'):
    cv2.imshow('Car Concentration And Security Assistant', quadro)
else:
    draw(quadro, guida_sicura, 100, 370) 
    cambia_colore(quadro, pixels, 150, 210, 190)

In questa porzione di codice si entra in un nuovo ciclo **while** finchè non si preme su tastiera il tasto '**q**', le operazioni svolte sono:
* si acquisisce l'immagine dalla webcam tramite la funzione **read()**;
* si applica il filtro **BRG2GRAY** della libreria **CV2** all'immagine acquisita per poter utilizzare i classificatori;
* si utilizza il classificatore della faccia, il quale restituisce una lista pari al numero di facce trovate e ogni elemento di questa contiene un'ulteriore lista di quattro elementi: posizione verticale, posizione orizzontale, larghezza e altezza della regione di interesse;
* se la lista **faccia** non è vuota si cercano gli attributi della faccia rilevata più estesa (tramite la funzione **main_face** definita precedemtemente) e si riquadra l'area contenente tale volto con l'apposita funzione di **cv2**;
* all'interno della sola regione di interesse (dove è presente la faccia) si procede alla ricerca degli occhi;
* si invoca la funzione **check** per aggiornare lo stato degli occhi passandole in ingresso il numero di occhi rilevati;
* ogni occhio viene riquadrato;
* sul quadro vengono sovrascritte le immagini che segnalano la *guida sicura* e gli *occhi aperti* e si mette in pausa l'audio relativo alla distrazione **avviso2**

![occhi aperti](../Assets/Img/occhi_aperti.png)

* se, invece, la lista **faccia** è vuota viene eseguito solo il ramo **else** e si si genera un avviso di distrazione visivo (immagini sul quadro) e sonoro (si avvia audio):

![Distrazione](../Assets/Img/distrazione.png)

![Triangolo giallo](../Assets/Img/attenzione_giallo.png)


In [ ]:
while cv2.waitKey(30) != ord('q'):
    ret, camera = cap.read()
        
    gray = cv2.cvtColor(camera, cv2.COLOR_BGR2GRAY)
    faccia = face_cascade.detectMultiScale(gray, 1.3, 5)   
    
    
    if len(faccia) != 0:
        fx, fy, fw, fh = main_face(faccia)
        
        cv2.rectangle(camera,(fx,fy),(fx+fw,fy+fh),(150,210,190), 1)   
        
        roi_gray = gray[fy:fy+fh, fx:fx+fw]
        roi_color = camera[fy:fy+fh, fx:fx+fw]
        
        occhi = eye_cascade.detectMultiScale(roi_gray, maxSize=(int(fw/5),int(fh/5)), minSize=(int(fw/7),int(fh/7)))
        
        check(len(occhi))
        
        for [ex,ey,ew,eh] in occhi:                                   
            cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh), (150,210,190), 1)
        
        avviso2.pause()
        draw(quadro, guida_sicura, 100, 370)
        draw(quadro, occhi_aperti, 500, 455)
        
    else:
        draw(quadro, guida_distratta, 100, 370)
        draw(quadro, triangolo_giallo, 500, 455)
        avviso2.play()

A seguire si verifica lo stato occhi e si aggiorna opportunamente l'output:

* se nel dizionario **'stato_occhi'** la variabile **occhi_chiusi** è Vera si genera un avviso adeguato visivo (immagini sul quadro) e sonoro (si avvia audio):

![sonno](../Assets/Img/Allarme_sonno.png) ![pericolo](../Assets/Img/Pericolo.png)

altrimenti viene messo in pausa l'audio relativo agli occhi chiusi (**avviso**).


In [ ]:
    if stato_occhi['occhi_chiusi']:
        avviso.play()
        draw(quadro, pericolo, 100, 370)
        draw(quadro, triangolo, 500, 455)
    else:
        avviso.pause()

Alla fine del ciclo si sovrappone all'immagine del quadro quella ricevuta in input dalla camera, con faccia e occhi eventualmente riquadrati, e viene mostrato a video l'immagine **quadro**.

In [ ]:
    draw(quadro, camera, 235, 375)
    cv2.imshow('Car Concentration And Security Assistant', quadro)

## Terminazione

Al termine del ciclo precedente, avvenuto premendo da tastiera il tasto '**q**', vengono messi in pausa gli avvisi, nel caso in cui fossero in fase di *play*, e chiuse tutte le finestre aperte.

In [ ]:
avviso.pause()
avviso2.pause()
        
cap.release()
cv2.destroyAllWindows()

## Fonti 

* Sito [OpenCV](http://docs.opencv.org/3.0-beta/doc/py_tutorials/py_gui/py_image_display/py_image_display.html) per la consultazione della documentazione sulle funzioni della libreria *cv2*;
* Download dei [classificatori](https://github.com/opencv/opencv/tree/master/data/haarcascades)
* Funzionamento algoritmi di tipo Cascade per la [Face Detection](http://docs.opencv.org/master/d7/d8b/tutorial_py_face_detection.html#gsc.tab=0)